Imports

In [91]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
import random
import pandas as pd
import math
# import scipy.sparse as sp

In [92]:
input = "input.xlsm"
# input = "https://umontevideo-my.sharepoint.com/personal/svilardelvalle_um_edu_uy/Documents/Horarios_2025/input.xlsm"

In [93]:
# clases
from entities import Dia, Horario, BloqueHorario, Profesor, Materia, Grupo, Prioridad, Superposicion

In [94]:
# variables
from variables import u, v, w, x, y, z

# Functions

In [95]:
# cargar datos
"""
This module provides functions to manage and manipulate data related to 'materias', 'grupos', and 'profesores'.

Functions:
    add_materia(materias, nombre, carga_horaria, cantidad_dias, grupos=[], profesores=[], cantidad_profesores=1):
        Adds a new 'materia' to the list of 'materias'.
        
    add_grupo(grupos, anio, turno, carrera, particion, recurse, aux):
        Adds a new 'grupo' to the list of 'grupos'.
        
    add_profesor(profesores, nombre, minimizar_dias=False):
        Adds a new 'profesor' to the list of 'profesores'.
        
    lista_profesores(profesores, nombres):
        Returns a list of 'profesores' matching the given names.
        
    lista_grupos(grupos, nombres):
        Returns a list of 'grupos' matching the given names.
"""

# crear materia
def add_materia(materias, nombre, carga_horaria, cantidad_dias, grupos=[], profesores=[], cantidad_profesores=1):
    id = len(materias)
    try:
        carga_horaria = int(carga_horaria)
    except:
        carga_horaria = 0
    try:
        cantidad_dias = int(cantidad_dias)
    except:
        cantidad_dias = 0
    materias.append(Materia(id, nombre, carga_horaria=carga_horaria, cantidad_dias=cantidad_dias,
                            grupos=grupos, profesores=profesores, cantidad_profesores=cantidad_profesores))

# crear grupo
def add_grupo(grupos, anio, turno, carrera, particion, recurse, aux):
    id = len(grupos)
    grupos.append(Grupo(id, anio, turno, carrera, particion, bool(recurse), aux))

# crear profesor
def add_profesor(profesores, nombre, minimizar_dias=False):
    id = len(profesores)
    profesor = Profesor(id, nombre, minimizar_dias)
    if not profesor in profesores:
        profesores.append(profesor)

def lista_profesores(profesores, nombres):
    profs = []
    for n in nombres:
        for p in profesores:
            if str(p) == n:
                profs.append(p)
                break
    return profs

def lista_grupos(grupos, nombres):
    gs = []
    for n in nombres:
        for g in grupos:
            if str(g) == n:
                gs.append(g)
                break
    return gs


In [96]:
#prioridades

"""
Generates a fixed priority array for the given time blocks with the specified priority value.
Args:
    bloques_horario (list): A list of time blocks.
    value (int): The fixed priority value to be assigned to each time block.
Returns:
    list: A list of lists where each inner list contains a time block and the specified priority value.
"""


def update_no_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in profesor.no_disponible:
            profesor.no_disponible.append(bloques_horario[i])
"""
Updates the 'no_disponible' list of a professor by adding the specified time blocks.
Args:
    profesor (Profesor): The professor whose availability is being updated.
    bloques_horario (list): A list of time blocks.
    no_disp_index (list): A list of indices indicating which time blocks the professor is not available for.
"""


def update_prioridad(profesor, bloques_horario, array_prioridad):
    """
    Updates the priority list of a professor based on the given priority array.
    Args:
        profesor (Profesor): The professor whose priorities are being updated.
        bloques_horario (list): A list of time blocks.
        array_prioridad (list): A list of tuples where each tuple contains a time block index and a priority value.
    """
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp(profesor, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], profesor = profesor)
        if not (prior in profesor.prioridades):
            profesor.prioridades.append(prior)

def random_pr(bloques_horario):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, random.randint(0,3)])
    return pr_array
"""
Generates a random priority array for the given time blocks.
Args:
    bloques_horario (list): A list of time blocks.
Returns:
    list: A list of lists where each inner list contains a time block and a random priority value between 0 and 3.
"""


def fixed_pr(bloques_horario, value):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, value])
    return pr_array

"""
Generates a fixed priority array for the given time blocks with the specified priority value.
Args:
    bloques_horario (list): A list of time blocks.
    value (int): The fixed priority value to be assigned to each time block.
Returns:
    list: A list of lists where each inner list contains a time block and the specified priority value.
"""



def update_no_disp_mat(materia, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in materia.no_disponible:
            materia.no_disponible.append(bloques_horario[i])
"""
Updates the 'no_disponible' list of a subject by adding the specified time blocks.
Args:
    materia (Materia): The subject whose availability is being updated.
    bloques_horario (list): A list of time blocks.
    no_disp_index (list): A list of indices indicating which time blocks the subject is not available for.
"""


def update_prioridad_mat(materia, bloques_horario, array_prioridad):
    """
    Updates the priority list of a subject based on the given priority array.
    Args:
        materia (Materia): The subject whose priorities are being updated.
        bloques_horario (list): A list of time blocks.
        array_prioridad (list): A list of tuples where each tuple contains a time block index and a priority value.
    """
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp_mat(materia, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], materia = materia)
        if not (prior in materia.prioridades):
            materia.prioridades.append(prior)
    


In [97]:
#superposicion
"""
Calculate the overlap between two given objects.

This function checks if two objects, `m1` and `m2`, are the same or if they have any common groups.
It returns a `Superposicion` object indicating the type of overlap.

Parameters:
m1 (object): The first object to compare. It should have an attribute `grupos` which is a list of groups.
m2 (object): The second object to compare. It should have an attribute `grupos` which is a list of groups.

Returns:
Superposicion: An object representing the overlap. If `m1` and `m2` are the same, it returns `Superposicion(0, m1, m2)`.
               If they share any common groups, it returns `Superposicion(1, m1, m2)`. Otherwise, it returns `Superposicion(0, m1, m2)`.
"""

def calcular_super(m1, m2):
    if m1 == m2:
        return Superposicion(0, m1, m2)
    else:
        s = False
        for g1 in m1.grupos:
            if g1 in m2.grupos:
                s = True
                break
        return Superposicion(1 if s else 0, m1, m2)

In [98]:
#imprimir
def print_timetable(dias, horarios, u_dict, w_dict, grupos, anios):
    """
    Prints the timetable for given days, schedules, groups, and years.

    Args:
        dias (list): List of days to be included in the timetable.
        horarios (list): List of schedules to be included in the timetable.
        u_dict (dict): Dictionary containing information about subjects.
        w_dict (dict): Dictionary containing information about professors.
        grupos (list): List of groups to be included in the timetable.
        anios (list): List of years to be included in the timetable.

    """
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            print('\t', *[str(d) for d in dias], sep='\t\t')
            print('··········································································································')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for m in mats:
                    if m is None:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        # ps = search_profesor(m, w_dict)
                        # lista.append(str(m) + " " + str([str(p) for p in ps]))
                        lista.append(str(m))
                print(str(h), *lista, sep='\t\t')

def print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias):
    """
    Prints the timetable for each professor.

    Parameters:
    dias (list): List of days to include in the timetable.
    horarios (list): List of time slots to include in the timetable.
    u_dict (dict): Dictionary containing university-related data.
    w_dict (dict): Dictionary containing week-related data.
    profesores (list): List of professor objects.
    materias (list): List of subject objects.
    """
    for p in profesores:
        if len(intersection([m.nombre for m in materias], p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def intersection(array1, array2):
    ret = []
    for i in array1:
        if i in array2:
            ret.append(i)
    return ret

def print_prioridades(dias, horarios, profesores):
    for p in profesores:
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            prioridades = [str(search_prioridad(BloqueHorario(d,h),p)) for d in dias]
            print(str(h), *prioridades, sep='\t')

def print_prioridades_materia(dias, horarios, materia):
    """
    Prints the priorities for a given subject (materia) based on the provided days and schedules.

    Args:
        dias (list): A list of days.
        horarios (list): A list of schedules corresponding to the days.
        materia (object): An object representing the subject, which should have a 'profesores' attribute.

    """
    print("Materia: ", str(materia))    
    print_prioridades(dias, horarios, materia.profesores)

def search_materia(b, g, u_dict):
    """
    Searches for a 'materia' object in the given dictionary of 'u' objects that matches the specified criteria.

    Args:
        b (str): The schedule (horario) to match.
        g (str): The group (grupo) to match within the 'materia'.
        u_dict (dict): A dictionary where keys are 'u' object IDs and values are 'u' objects.

    Returns:
        materia: The 'materia' object that matches the specified schedule and group, and has a variable 'x' rounded to 1.
        None: If no matching 'materia' object is found.
    """
    for u_id in u_dict:
        u_obj = u_dict[u_id]
        if u_obj.horario == b and g in u_obj.materia.grupos and round(u_obj.variable.x) == 1:
            return u_obj.materia
    return None

def search_profesor(materia, w_dict):
    """
    Searches for professors assigned to a given subject based on a weight dictionary.
    Args:
        materia (object): The subject object which contains a list of professors.
        w_dict (dict): A dictionary with keys as tuples of (subject_id, professor_id) and values as objects 
                       containing a variable attribute with an x property.
    Returns:
        list: A list of professors assigned to the given subject.
    """
    
    if materia is None:
        return []
    
    ps = []
    for p in materia.profesores:
        if round(w_dict[materia.id, p.id].variable.x) == 1:
            ps.append(p)
    return ps

def search_profesor_by_nombre(profesores, nombre):
    for p in profesores:
        if p.nombre == nombre:
            return p
    return None

def search_materias_by_nombre(materias, nombre):
    ret = []
    for m in materias:
        if m.nombre == nombre:
            ret.append(m)
    return ret
    
def search_materia_prof(b, p, u_dict, w_dict, materias):
    """
    Searches for a 'materia' (subject) that matches the given block and professor.
    Args:
        b: The block to search within.
        p: The professor to search for.
        u_dict (dict): A dictionary where keys are 'materia' IDs and values are objects containing scheduling information.
        w_dict (dict): A dictionary where keys are tuples of 'materia' and professor IDs, and values are objects containing assignment information.
        materias (list): A list of 'materia' objects to search through.
    Returns:
        The 'materia' object that matches the given block and professor, or None if no match is found.
    """
    for m in materias:
        if round(w_dict[m.id, p.id].variable.x * u_dict[m.id, b.id()].variable.x) == 1:
            return m
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia

    # for u_id in u_dict:
    #     u_obj = u_dict[u_id]
    #     bloque = u_obj.horario
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia
    return None

def search_prioridad(b, p):
    if b in p.no_disponible:
        return "-"
    for pr in p.prioridades:
        if pr.bloque_horario == b:
            return pr.value
        
def grupos_anio(grupos, anio):
    """
    Filters a list of groups by a specific year.

    Args:
        grupos (list): A list of group objects, where each object has an 'anio' attribute.
        anio (int): The year to filter the groups by.

    Returns:
        list: A list of group objects that match the specified year.
    """
    gs = []
    for g in grupos:
        if g.anio == anio:
            gs.append(g)
    return gs

def horarios_turno(horarios, turno):
    """
    Filters a list of schedule objects based on a specified shift.

    Args:
        horarios (list): A list of schedule objects. Each object should have a 'turnos' attribute.
        turno (str): The shift to filter by.

    Returns:
        list: A list of schedule objects that include the specified shift.
    """
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h)
    return hs

def horarios_ids_turno(horarios, turno):
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h.id)
    return hs




In [99]:
# imprimir para mostrar en Excel
def print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, anios):
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            # print('\t', *[str(d) for d in dias], sep='\t\t')
            print('', *[str(d) for d in dias], sep='\t')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for m in mats:
                    if m is None:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        ps = search_profesor(m, w_dict)
                        lista.append(str(m) + " " + str([str(p) for p in ps]))
                        # lista.append(str(m))
                # print(str(h), *lista, sep='\t\t')
                print(str(h), *lista, sep='\t')

def print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias):
    for p in profesores:
        if len(intersection([m.nombre for m in materias], p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p))
        print('', *[str(d) for d in dias], sep='\t')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def print_timetable_salones(dias, horarios, niveles):
    print('\t', *[str(d) for d in dias], sep='\t')
    print('··········································································································')
    for h in horarios:
        lista = [str(niveles[d.id, h.id])+"%" for d in dias]
        print(str(h), *lista, sep='\t')




In [100]:
#copiar resultados de variables a hoja de excel

def copy_variables_excel(u_dict, w_dict):

    data_u = {
        'materia': [],
        'dia': [],
        'horario': [],
        'U': []
    }
    for u_i in u_dict:
        data_u['materia'].append(u_dict[u_i].materia.id)
        data_u['dia'].append(u_dict[u_i].horario.dia.id)
        data_u['horario'].append(u_dict[u_i].horario.horario.id)
        data_u['U'].append(round(u_dict[u_i].variable.x))

    data_w = {
        'materia': [],
        'profesor': [],
        'W': []
    }
    for w_i in w_dict:
        data_w['materia'].append(w_dict[w_i].materia.id)
        data_w['profesor'].append(w_dict[w_i].profesor.id)
        data_w['W'].append(round(w_dict[w_i].variable.x))

    with pd.ExcelWriter("output.xlsx") as writer:
        pd.DataFrame(data_u).to_excel(writer, sheet_name="u", index=False)
        pd.DataFrame(data_w).to_excel(writer, sheet_name="w", index=False)

    

In [101]:
#buscar materias
"""
This module provides functions to filter and group subjects (materias) based on professors and groups.
Functions:
    materias_profesor(profesor, materias_total):
        Filters and returns a list of subjects taught by a specific professor.
    agrupar_materias(lista_materias):
        Groups subjects by their string representation and returns a dictionary where keys are subject names and values are lists of subjects.
    materias_grupo(grupo, materias_total):
        Filters and returns a list of subjects that belong to a specific group.
    materias_grupo_ids(grupo, materias_total):
        Filters and returns a list of subject IDs that belong to a specific group.
"""

def materias_profesor(profesor, materias_total):
    mats = []

    for m in materias_total:
        if profesor in m.profesores:
            mats.append(m)

    return mats

def agrupar_materias(lista_materias):
    lista_nombres = {}

    for m in lista_materias:
        if str(m) not in lista_nombres:
            lista_nombres[str(m)] = [m]
        else:
            lista_nombres[str(m)].append(m)

    return lista_nombres

def materias_grupo(grupo, materias_total):
    materias = []

    for m in materias_total:
        if grupo is not None and grupo in m.grupos:
            materias.append(m)
            
    return materias

def materias_grupo_ids(grupo, materias_total):
    materias_ids = []

    for m in materias_grupo(grupo, materias_total):
        if grupo is not None and grupo in m.grupos:
            materias_ids.append(m.id)
            
    return materias_ids


# Formulacion

## (1) Datos / Constantes

### Horarios

In [102]:
"""
    This script initializes and processes scheduling data for a university timetable.

    Classes:
        Dia: Represents a day of the week with an ID and name.
        Horario: Represents a time slot with an ID, start time, end time, and a list of periods.
        BloqueHorario: Represents a block of time on a specific day.

    Variables:
        dias (list of Dia): A list of days of the week.
        dias_ids (list of int): A list of IDs corresponding to the days of the week.
        horarios (list of Horario): A list of time slots.
        horarios_ids (list of int): A list of IDs corresponding to the time slots.
        bloques_horario (dict of tuple: BloqueHorario): A dictionary mapping (day ID, time slot ID) to BloqueHorario objects.
        bloques_horario_ids (list of tuple): A list of tuples representing the keys of the bloques_horario dictionary.
    """

dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)


In [103]:
horarios = []

df = pd.read_excel(input, sheet_name="turnos")

for i in range(len(df)):
    id = df["#"][i]
    inicio = df["Inicio"][i]
    fin = df["Fin"][i]

    turnos = []
    t = 1
    while True:
        try:
            new_t = df["Turno " + str(t)][i]
            if isinstance(new_t, str):
                turnos.append(new_t)
            t += 1
        except:
            break

    try:
        id = int(id)
        horarios.append(Horario(id, inicio.strftime("%H:%M"), fin.strftime("%H:%M"), turnos))
    except:
        pass


# horarios = [
#     Horario(1, "8:00", "8:50", ["m"]),
#     Horario(2, "8:50", "9:40", ["m"]),
#     Horario(3, "9:50", "10:40", ["m"]),
#     Horario(4, "10:40", "11:30", ["m"]),
#     Horario(5, "11:40", "12:30", ["m"]),
#     Horario(6, "12:30", "13:20", ["m"]),
#     Horario(7, "14:00", "14:50", ["t"]),
#     Horario(8, "14:50", "15:40", ["t"]),
#     Horario(9, "15:50", "16:40", ["t"]),
#     Horario(10, "16:40", "17:30", ["t"])
# ]

horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)

bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)


PermissionError: [Errno 13] Permission denied: 'input.xlsm'

### Profesores

In [ ]:
profesores = []

df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]
    # # grupos_max = df["Max grupos"][i]
    # if not isinstance(grupos_max, int):
    #     grupos_max = None
    minimizar_dias = isinstance(df["Minimizar dias"][i], str)
    # if str(cantidad_dias) == "nan":
    #     cantidad_dias = None
    # else:
    #     cantidad_dias = int(cantidad_dias)
        
    if isinstance(nombre, str):
        add_profesor(profesores, nombre, bool(minimizar_dias))

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
print(*[p for p in profesores], sep=", ")

ac, aca, ad, adr, am, ame, ap, apa, apr, as, cm, cp, cr, db, dc, dd, dj, dl, dm, ds, dv, ec, epe, es, fb, gaa, gb, gba, gc, gd, gq, gs, guc, hf, hr, jam, jcc, jd, jds, jf, jj, jk, jl, jpa, jpe, jpf, kb, lc, le, lg, mb, mbo, mg, mjc, mp, mpi, mpr, mr, mro, mz, nd, nn, ns, pn, po, ps, pu, rc, rl, rr, rs, sb, sgp, sp, st, svi, svv, th, vt


### Prioridades

In [ ]:
# prioridades fijas

# for p in profesores:
#     update_prioridad(p, bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
    

In [ ]:
# prioridades desde excel: 1 profesor por hoja
"""
Processes scheduling priorities for professors from an Excel file.
For each professor in the `profesores` list, this function attempts to read their
scheduling priorities from a specified Excel file. If the sheet corresponding to
the professor does not exist, it assigns random priorities. Otherwise, it extracts
the scheduling priorities from the Excel sheet and updates the professor's priorities.
Steps:
1. Attempt to read the Excel sheet corresponding to the professor.
2. If the sheet does not exist, assign random priorities and continue to the next professor.
3. If the sheet exists, extract the scheduling priorities.
4. Update the professor's scheduling priorities with the extracted data.
Parameters:
- profesores (list): List of professor identifiers.
- bloques_horario (list): List of available time blocks.
- pd (module): Pandas module for reading Excel files.
- update_prioridad (function): Function to update the professor's scheduling priorities.
- random_pr (function): Function to generate random priorities for a professor.
Excel File Structure:
- Each sheet corresponds to a professor.
- The first column contains time block identifiers.
- Subsequent columns contain priority values for each time block.
Example:
    profesores = ['ProfA', 'ProfB']
    bloques_horario = ['Block1', 'Block2']
    update_prioridad = some_update_function
    random_pr = some_random_function
    # Call the function with the above parameters
"""


for prof in profesores:
    
    try:
        df = pd.read_excel(input, sheet_name=str(prof))
    except:
        # update_prioridad(prof, bloques_horario, random_pr(bloques_horario))
        update_prioridad(p, bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
        continue

    array_prioridad = []
    # array_prioridad[i] = [(d,h),a]

    h_id = []

    for heading in df:

        if heading == str(prof):
            for row in df[heading]:
                h_id.append(row)
        else:
            i = 0
            for row in df[heading]:
                array_prioridad.append([(heading, h_id[i]), row])
                i += 1

    # print(array_prioridad)
    
    update_prioridad(prof, bloques_horario, array_prioridad)


### Grupos

In [ ]:
#grupos
"""
This script processes an Excel file containing group information and populates a list of group objects.
Functions:
    add_grupo(grupos, *args): Adds a group to the list of groups.
Variables:
    grupos (list): A list to store group objects.
    df (DataFrame): DataFrame containing the data read from the Excel file.
    grupos_ids (list): A list to store the IDs of the groups.
Workflow:
1. Read the Excel file input from the sheet named "grupos" into a DataFrame.
2. Iterate over each row in the DataFrame and extract relevant information.
3. Append extracted information to a list of arguments.
4. Replace 'nan' values with None and attempt to convert arguments to integers.
5. Call the add_grupo function with the list of arguments to add a group to the grupos list.
6. Populate the grupos_ids list with the IDs of the groups.
"""
grupos = []


df = pd.read_excel(input, sheet_name="grupos")

for i in range(len(df)):
    
    args = []
    args.append(df["Año"][i])  # anio
    args.append(df["Turno"][i])     # turno
    args.append(df["Carrera"][i])   # carrera
    args.append(df["Particion"][i])  # particion
    args.append(isinstance(df["Recurse"][i], str)) # recurse
    args.append(df["Auxiliar"][i])

    for i in range(0, len(args)):
        
        if str(args[i]) == "nan":
            args[i] = None

        try:
            args[i] = int(args[i])
        except:
            pass      

    add_grupo(grupos, *args)

grupos_ids = []
for g in grupos:
    grupos_ids.append(g.id)


c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
print(*[str(g) for g in grupos])

11 12 13 14 1LIC 1REC1 1REC2 2CIV 2IND 2INF1 2INF2 2TEL 2LIC 3CIV 3IND 3TEL 3INF1 3INF2 3LIC 4CIV 4IND 4TEL 4INF1 4INF2 4LIC1 4LIC2 5CIV 5IND 5TEL 5INF dummy


In [ ]:
anios = []
"""
This script processes a list of groups and extracts unique years from them.

Variables:
    anios (list): A list to store unique years extracted from the groups.
    grupos (list): A list of group objects, each containing an 'anio' attribute.

The script iterates through each group in 'grupos', checks if the 'anio' attribute is not None and not already in the 'anios' list, and appends it to 'anios' if both conditions are met.
"""
for g in grupos:
    if not g.anio is None and not g.anio in anios:
            anios.append(g.anio)


### Materias 

In [ ]:
#materias
"""
This script processes an Excel file containing information about various subjects (materias) and extracts relevant details to create a list of subjects.
The script performs the following steps:
1. Reads the Excel file input from the sheet named "materias".
2. Iterates through each row of the DataFrame to extract subject details.
3. For each subject, it extracts the name, hourly load, number of days per week, and the number of professors.
4. It then attempts to extract the names of the professors and groups associated with each subject.
5. Finally, it adds the subject to the list of subjects using the `add_materia` function.
Variables:
- materias: A list to store the details of each subject.
- df: DataFrame containing the data read from the Excel file.
- nombre: The name of the subject.
- carga_horaria: The hourly load of the subject.
- cantidad_dias: The number of days per week the subject is taught.
- cantidad_profesores: The number of professors teaching the subject.
- profs: A list to store the names of the professors.
- gs: A list to store the names of the groups.
Functions:
- add_materia: Adds a subject to the list of subjects.
- lista_grupos: Processes and returns a list of groups.
- lista_profesores: Processes and returns a list of professors.
"""

materias = []

df = pd.read_excel(input, sheet_name="materias")

for i in range(len(df)):

    nombre = df["Materia"][i]
    if not isinstance(nombre, str):
        continue # cancel if no name is provided

    carga_horaria = df["Carga Horaria"][i]
    cantidad_dias = df["Días por semana"][i]
    
    try:
        cantidad_profesores = int(df["Cantidad profesores"][i])
    except:
        cantidad_profesores = 1 # default value

    profs = []
    p = 1
    while True:
        try:
            new_p = df["Profesor " + str(p)][i]
            if isinstance(new_p, str):
                profs.append(new_p)
            p += 1
        except:
            break

    gs = []
    g = 1
    while True:
        try:
            new_g = df["Grupo " + str(g)][i]
            if isinstance(new_g, str):
                gs.append(new_g)
            g += 1
        except:
            break

    
    
    # gs = lista_grupos(grupos, [x.strip() for x in str(df["Grupos"][i]).split(',')])
    # profs = lista_profesores(profesores, [x.strip() for x in str(df["Profesores"][i]).split(',')])

    add_materia(materias, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs), cantidad_profesores)



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Filtros

In [ ]:
# filtrar por turno
# materias = [m for m in materias if "m" in m.turnos()]
# print(*horarios)

In [ ]:
# filtrar por anio
# anio = 4
# materias = [m for m in materias if anio in m.anios()]
# grupos = [g for g in grupos if g.anio == anio]

# ms = []
# gs = []

# for a in [1,2,3,4]:

#     for m in materias:
#         if a in m.anios():
#             ms.append(m)
#     for g in grupos:
#         if g.anio == a:
#             gs.append(g)


# materias = ms
# grupos = gs

# print(*grupos)


In [ ]:
print(*grupos)

11 12 13 14 1LIC 1REC1 1REC2 2CIV 2IND 2INF1 2INF2 2TEL 2LIC 3CIV 3IND 3TEL 3INF1 3INF2 3LIC 4CIV 4IND 4TEL 4INF1 4INF2 4LIC1 4LIC2 5CIV 5IND 5TEL 5INF dummy


In [ ]:
materias_ids = []
# materias_dict = {}
for m in materias:
    materias_ids.append(m.id)
    # materias_dict[m.id] = m


In [ ]:
# materias por profesor:
"""
This script processes an Excel file containing information about professors and their respective subjects.
It reads the data from the specified sheet and updates the list of subjects for each professor.
Steps:
1. Load the Excel file "input.xlsm" and read the sheet named "profesores" into a DataFrame.
2. Iterate through each row of the DataFrame to extract professor names and their subjects.
3. For each professor, gather the subjects they teach along with the maximum number of groups for each subject.
4. Update the professor's list of subjects using the `search_profesor_by_nombre` function.
Variables:
- df: DataFrame containing the data from the Excel sheet.
- nombre: Name of the professor.
- lista_mats: List of dictionaries, each containing the subject name and the maximum number of groups.
- m: Counter for the subject columns in the DataFrame.
- new_m: Name of the subject being processed.
- grupos_max: Maximum number of groups for the subject.
- prof: Professor object found using the `search_profesor_by_nombre` function.
Functions:
- search_profesor_by_nombre(profesores, nombre): Searches for a professor by name in the given list of professors.
Note:
- The script assumes that the Excel file and sheet structure are consistent with the expected format.
- The `search_profesor_by_nombre` function and other commented-out functions need to be defined elsewhere in the codebase.
"""
df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]

    lista_mats = []
    m = 1
    while True:
        try:
            new_m = df["Materia " + str(m)][i]
            
            if isinstance(new_m, str):
                grupos_max = int(df["Grupos materia " + str(m)][i])
                lista_mats.append({"nombre_materia" : new_m, "grupos_max" : grupos_max})
            m += 1
        except:
            break
    
    prof = search_profesor_by_nombre(profesores, nombre)
    if not prof is None:
        prof.lista_materias = lista_mats


    # add_materia(materias, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs))


c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# for p in profesores:
#     mats = ""
#     for m in materias:
#         if p in m.profesores:
#             mats += str(m) + " "
#     print(p, ":", mats)



In [ ]:
# print(*[[p.lista_materias, p.nombre] for p in profesores], sep="\n")

### Superposicion

In [ ]:
# superposicion
"""
Calculates the overlap between pairs of subjects and stores the results in a dictionary.

This script iterates over all pairs of subjects in the `materias` list and calculates the overlap 
between each pair using the `calcular_super` function. The results are stored in the `superposicion` 
dictionary, where the keys are tuples of subject IDs and the values are the calculated overlaps.

Variables:
    superposicion (dict): A dictionary to store the overlap results between pairs of subjects.
    materias (list): A list of subject objects, each with an `id` attribute.

Functions:
    calcular_super(m1, m2): A function that takes two subject objects and returns the overlap between them.

Dictionary Structure:
    superposicion[(m1.id, m2.id)] = overlap_value
"""
superposicion = {}

for m1 in materias:
    for m2 in materias:
        superposicion[(m1.id, m2.id)] = calcular_super(m1, m2)


In [ ]:
print_prioridades(dias, horarios, profesores)


 Profesor:  ac
		lun	mar	mie	jue	vie
·····················································
08:00-08:50	None	None	None	None	None
08:50-09:40	None	None	None	None	None
09:50-10:40	None	None	None	None	None
10:40-11:30	None	None	None	None	None
11:40-12:30	None	None	None	None	None
12:30-13:20	None	None	None	None	None
14:10-15:00	None	None	None	None	None
15:10-16:00	None	None	None	None	None
16:10-17:00	None	None	None	None	None
17:00-17:50	None	None	None	None	None
18:00-18:50	None	None	None	None	None
18:50-19:40	None	None	None	None	None
19:50-20:40	None	None	None	None	None
20:40-21:30	None	None	None	None	None
21:40-22:30	None	None	None	None	None

 Profesor:  aca
		lun	mar	mie	jue	vie
·····················································
08:00-08:50	None	None	None	None	None
08:50-09:40	None	None	None	None	None
09:50-10:40	None	None	None	None	None
10:40-11:30	None	None	None	None	None
11:40-12:30	None	None	None	None	None
12:30-13:20	None	None	None	None	None
14:10-15:00	None	None	None	None	None
1

In [ ]:
# prioridades de materia

# prioridades desde excel: 1 profesor por hoja

# for mat in materias:
    
#     try:
#         df = pd.read_excel(input, sheet_name=str(mat))
#     except:
#         # update_prioridad_mat(mat, bloques_horario, random_pr(bloques_horario))
#         continue

#     array_prioridad = []
#     # array_prioridad[i] = [(d,h),a]

#     h_id = []

#     for heading in df:

#         if heading == str(mat):
#             for row in df[heading]:
#                 h_id.append(row)
#         else:
#             i = 0
#             for row in df[heading]:
#                 array_prioridad.append([(heading, h_id[i]), row])
#                 i += 1

#     # print(array_prioridad)
#     update_prioridad_mat(mat, bloques_horario, array_prioridad)
#     update_prioridad()

In [ ]:
print("Grupos:", len(grupos))
# print([str(i) for i in grupos])
print("Materias:", len(materias))
# print([str(i) for i in materias])
print("Profesores:", len(profesores))
# print([str(i) for i in profesores])

Grupos: 31
Materias: 122
Profesores: 79


## (2) Variables


$$ u_{mb} \in \{0,1\} $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} \in \{0,1\} $$

    m: materia
    d: dia

$$ w_{mp} \in \{0,1\} $$

    m: materia
    p: profesor


$$ x_{gb} \in \{0,1\} $$

    g: grupo
    b: bloque horario = [dh]: dia y hora

$$ y_{pb} \in \{0,1\} $$

    p: profesor
    b: bloque horario = [dh]: dia y hora

$$ z_{pd} \in \{0,1\} $$

    p: profesor
    d: dia

In [ ]:
# variables
"""
This script initializes and populates several dictionaries to map combinations of entities 
(materias, bloques_horario, dias, profesores, and grupos) to corresponding function outputs.
Variables:
    u_dict (dict): Maps tuples of (materia.id, bloque_horario) to the result of function u(m, bloques_horario[b_id]).
    v_dict (dict): Maps tuples of (materia.id, dia.id) to the result of function v(m, d).
    w_dict (dict): Maps tuples of (materia.id, profesor.id) to the result of function w(m, p).
    x_dict (dict): Maps tuples of (grupo.id, bloque_horario) to the result of function x(g, bloques_horario[b_id]).
    y_dict (dict): Maps tuples of (profesor.id, bloque_horario) to the result of function y(p, bloques_horario[b_id]).
    z_dict (dict): Maps tuples of (profesor.id, dia.id) to the result of function z(p, d).
Prints:
    The length of each dictionary after it has been populated.
"""

u_dict = {}
for m in materias:   
    for b_id in bloques_horario:
        u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])
print("u: ", len(u_dict))

# print([str(us) for us in u_dict])

v_dict = {}
for m in materias:
    for d in dias:
        v_dict[(m.id, d.id)] = v(m, d)
print("v: ", len(v_dict))

w_dict = {}
for m in materias:
    for p in profesores:
        w_dict[(m.id, p.id)] = w(m, p)
print("w: ", len(w_dict))

x_dict = {}
for g in grupos:
    for b_id in bloques_horario:
        x_dict[(g.id, b_id)] = x(g, bloques_horario[b_id])
print("x: ", len(x_dict))

y_dict = {}
for p in profesores:
    for b_id in bloques_horario:
        y_dict[(p.id, b_id)] = y(p, bloques_horario[b_id])
print("y: ", len(y_dict))

z_dict = {}
for p in profesores:
    for d in dias:
        z_dict[(p.id, d.id)] = z(p, d)
print("z: ", len(z_dict))

u:  9150
v:  610
w:  9638
x:  2325
y:  5925
z:  395


In [ ]:
print(len(materias))
print(len(bloques_horario))

122
75


In [ ]:
# Create a new model
model = gp.Model("timetable")

Set parameter ServerTimeout to value 30
Set parameter TokenServer to value "10.4.0.1"


In [ ]:
# Create variables
"""
This script creates binary decision variables for multiple dictionaries using the Gurobi optimization model.
Variables:
    u_dict (dict): Dictionary containing elements for which binary variables "u" are created.
    v_dict (dict): Dictionary containing elements for which binary variables "v" are created.
    w_dict (dict): Dictionary containing elements for which binary variables "w" are created.
    x_dict (dict): Dictionary containing elements for which binary variables "x" are created.
    y_dict (dict): Dictionary containing elements for which binary variables "y" are created.
    z_dict (dict): Dictionary containing elements for which binary variables "z" are created.
Each element in the dictionaries is assigned a binary variable using the Gurobi model's `addVar` method.
"""

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

for u_i in u_dict: # crear variables "u" a partir de u_dict
    u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

for v_i in v_dict: # crear variables "v" a partir de v_dict
    v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

for w_i in w_dict: # crear variables "v" a partir de v_dict
    w_dict[w_i].variable = model.addVar(vtype=GRB.BINARY, name=str(w_dict[w_i]))


for x_i in x_dict: # crear variables "x" a partir de x_dict
    x_dict[x_i].variable = model.addVar(vtype=GRB.BINARY, name=str(x_dict[x_i]))

for y_i in y_dict: # crear variables "y" a partir de y_dict
    y_dict[y_i].variable = model.addVar(vtype=GRB.BINARY, name=str(y_dict[y_i]))

for z_i in z_dict: # crear variables "z" a partir de z_dict
    z_dict[z_i].variable = model.addVar(vtype=GRB.BINARY, name=str(z_dict[z_i]))


## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion (redundante con la definicion de la variable x)
$$ \sum_{b} \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$


In [ ]:
"""
    This code defines constraints to ensure that no two subjects (materias) overlap
    in the same time block (bloque horario).
    Constraints:
        1. For each time block (b) in bloques_horario_ids, the sum of the product of the decision variables 
        (u_dict[m1, b].variable and u_dict[m2, b].variable) and the overlap value (superposicion[(m1, m2)].value)
        for all pairs of subjects (m1, m2) must be equal to 0.
"""

model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
                            for m1 in materias_ids for m2 in materias_ids)
                == 0 for b in bloques_horario_ids)


## for g in grupos:
##     mats = materias_grupo(g, materias)
##     if len(mats) > 1:
##         # model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)
##         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)


{(1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (1, 6): <gurobi.QConstr Not Yet Added>,
 (1, 7): <gurobi.QConstr Not Yet Added>,
 (1, 8): <gurobi.QConstr Not Yet Added>,
 (1, 9): <gurobi.QConstr Not Yet Added>,
 (1, 10): <gurobi.QConstr Not Yet Added>,
 (1, 11): <gurobi.QConstr Not Yet Added>,
 (1, 12): <gurobi.QConstr Not Yet Added>,
 (1, 13): <gurobi.QConstr Not Yet Added>,
 (1, 14): <gurobi.QConstr Not Yet Added>,
 (1, 15): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (2, 6): <gurobi.QConstr Not Yet Added>,
 (2, 7): <gurobi.QConstr Not Yet Added>,
 (2, 8): <gurobi.QConstr Not Yet Added>,
 (2, 9): <gurobi.QConstr Not Yet Added>,
 (2, 10): 

In [ ]:
# print(*[[str(m), m.cantidad_dias] for m in materias], sep="\n")

#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [ ]:
for m in materias:
    model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) particion de horas por materia

##### (3.1.3.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [ ]:
for m in materias:
    model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.3.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [ ]:
for m in materias:

    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
    
    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)
    
    


##### (3.1.3.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turnos
    para todo m



In [ ]:
for m in materias:
    # no_bloques_materia_ids = [i for i in bloques_horario if not bloques_horario[i].horario.turno in m.turnos()]
    no_bloques_materia_ids = []
    for b in bloques_horario:
        if len([t for t in bloques_horario[b].horario.turnos if t in m.turnos()]) == 0:
            no_bloques_materia_ids.append(b)
    model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.4) horas consecutivas dentro de un dia

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [ ]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 1): <gurobi.QConstr Not Yet Added>,
 (4, 2): <gurobi.QConstr Not Yet Added>,
 (4, 3): <gurobi.QConstr Not Yet Added>,
 (4, 4): <gurobi.QConstr Not Yet Added>,
 (4, 5): <gurobi

#### (3.1.5) evitar dias consecutivos para una misma materia

$$ \sum_m \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$


In [ ]:
model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                 for m in materias_ids)


{0: <gurobi.QConstr Not Yet Added>,
 1: <gurobi.QConstr Not Yet Added>,
 2: <gurobi.QConstr Not Yet Added>,
 3: <gurobi.QConstr Not Yet Added>,
 4: <gurobi.QConstr Not Yet Added>,
 5: <gurobi.QConstr Not Yet Added>,
 6: <gurobi.QConstr Not Yet Added>,
 7: <gurobi.QConstr Not Yet Added>,
 8: <gurobi.QConstr Not Yet Added>,
 9: <gurobi.QConstr Not Yet Added>,
 10: <gurobi.QConstr Not Yet Added>,
 11: <gurobi.QConstr Not Yet Added>,
 12: <gurobi.QConstr Not Yet Added>,
 13: <gurobi.QConstr Not Yet Added>,
 14: <gurobi.QConstr Not Yet Added>,
 15: <gurobi.QConstr Not Yet Added>,
 16: <gurobi.QConstr Not Yet Added>,
 17: <gurobi.QConstr Not Yet Added>,
 18: <gurobi.QConstr Not Yet Added>,
 19: <gurobi.QConstr Not Yet Added>,
 20: <gurobi.QConstr Not Yet Added>,
 21: <gurobi.QConstr Not Yet Added>,
 22: <gurobi.QConstr Not Yet Added>,
 23: <gurobi.QConstr Not Yet Added>,
 24: <gurobi.QConstr Not Yet Added>,
 25: <gurobi.QConstr Not Yet Added>,
 26: <gurobi.QConstr Not Yet Added>,
 27: <gurob

#### (3.1.6) indisponibilidad de materia

$$ u_{mb} = 0 $$

    para todo m sin disponibilidad en b

In [ ]:
for m in materias:
    for b in m.no_disponible:
        model.addConstr(u_dict[m.id, b.id()].variable == 0)

### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ \sum_m w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [ ]:
for m in materias:
    for p in m.profesores:
        for b in p.no_disponible:
            model.addConstr(u_dict[m.id, b.id()].variable * w_dict[m.id, p.id].variable == 0)

LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ y_{pb} = 0 $$

    para todo p sin disponibilidad en b

In [ ]:
for p in profesores:
    for b in p.no_disponible:
        model.addConstr(y_dict[p.id, b.id()].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

NO ES NECESARIA (SE ASEGURA CON LA DEFINICION DE y_pb)

In [ ]:
for p in profesores:
    model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 1 for b in bloques_horario_ids)

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [ ]:
for m in materias:
    for p in profesores:
        if p not in m.profesores:
            model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) cantidad de profesores para una misma materia

$$ \sum_p {w_{mp}} = P_m $$

    para todo m

In [ ]:
for m in materias:
    model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == m.cantidad_profesores)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} \leq K_p $$

    para todo p



In [ ]:
# separar por nombre de materia
for p in profesores:
    for l in p.lista_materias:
        mats = search_materias_by_nombre(materias, l["nombre_materia"])
        model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

#### (3.2.5) definicion de variable "y" (asignacion horario-profesor)

$$ y_{pb} = \sum_m{u_{mb}·w_{mp}} $$

    para todo p, b

In [ ]:
model.addConstrs(y_dict[p, b].variable ==
                 gp.quicksum(u_dict[m,b].variable * w_dict[m,p].variable for m in materias_ids)
                 for b in bloques_horario_ids for p in profesores_ids)

{((1, 1), 0): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 1): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 2): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 3): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 4): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 5): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 6): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 7): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 8): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 9): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 10): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 11): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 12): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 13): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 14): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 15): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 16): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 17): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 18): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 19): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 20): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 21): <gurobi.

#### (3.2.6) definicion de variable "z" (para dias con clase por profesor)

$$ z_{pd} = OR_h{y_{p(dh)}} $$

    para todo p, d

In [ ]:
model.addConstrs(z_dict[p,d].variable ==
                gp.or_(y_dict[p,(d,h)].variable for h in horarios_ids)
                for p in profesores_ids for d in dias_ids)

{(0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 4): <gurobi.GenConstr *Awaiting Model Upd

Posible linealizacion:

$$ z_{pd} \leq \sum_h {y_{pdh}} \leq M \times z_{pd} $$

    para todo m, d


In [ ]:
# for p in profesores:

#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      <= 10 * z_dict[p.id,d].variable for d in dias_ids)
    
#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      >= z_dict[p.id,d].variable for d in dias_ids)

#### (3.2.7) limitar la cantidad de horas diarias por profesor (NO VA)


$$ \sum_{m,h} {w_{mp}·u_{mdh}} \leq {HD}_p $$

    para todo p, d


In [ ]:
# FALTA CHEQUEAR

# for p in profesores:
#     if not p.horas_max is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstrs(gp.quicksum(
#             w_dict[m, p.id].variable * u_dict[m,(d,h)].variable for m in materias_ids for h in horarios_ids
#             ) <= p.horas_max for d in dias_ids)

#### (3.2.8) limitar dias con clase por docente (NO VA COMO RESTRICCION)


$$ \sum_{m,d} {w_{mp}·v_{md}} \leq {Dmax}_p $$
$$ \sum_{d} {z_{pd}} \leq {Dmax}_p $$

    para todo p


In [ ]:
# MAL IMPLEMENTADA

# for p in profesores:
#     if not p.cantidad_dias is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstr(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ) <= p.cantidad_dias)
        

In [ ]:
# p = search_profesor_by_nombre(profesores, "jf")
# print(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ).getValue())

### (3.3) Restricciones externas

#### (3.3.1) cantidad de salones

$$ \sum_{m} {u_{mb}} \leq K $$

    para todo b

In [ ]:
K = 13      # 13 salones en este caso

model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= K for b in bloques_horario_ids)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Upda

#### (3.3.2) fijar horas manualmente

In [ ]:
# no coincidir QC y DBD3
try:
    qc = search_materias_by_nombre(materias, "QC")[0]
    dbd3 = search_materias_by_nombre(materias, "DBD3")[0]

    model.addConstr(gp.quicksum(u_dict[qc.id, b].variable * u_dict[dbd3.id, b].variable for b in bloques_horario_ids) == 0)
except:
    pass

In [ ]:
# no coincidir BPMN y EthHck
try:
    bpmn = search_materias_by_nombre(materias, "BPMN")[0]
    ethhck = search_materias_by_nombre(materias, "EthHck")[0]

    model.addConstr(gp.quicksum(u_dict[bpmn.id, b].variable * u_dict[ethhck.id, b].variable for b in bloques_horario_ids) == 0)
except:
    pass

In [ ]:
# fijar MDp al ultimo bloque
for mdp in search_materias_by_nombre(materias, "MDp"):
    if mdp.carga_horaria == 1:
        model.addConstr(gp.quicksum(u_dict[mdp.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-1]) == 0)

    


#### (3.3.3) practico despues del teorico

$$ \max_b\{(-b)*u_{mb}\} > \max_b\{(-b)*u_{m'b}\} $$

    para todo par (teorico, practico) =  (m, m')


In [ ]:
teo = []    # array con IDS de materias de teorico
"""
Variables:
    teo (list): Array containing IDs of theoretical subjects.
    prac (list): Array containing IDs of practical subjects.
    vars1erHora (dict): Dictionary mapping subject IDs to Gurobi integer variables representing the first hour.
    scaled_u_dict (dict): Dictionary mapping tuples of subject IDs and block IDs to scaled Gurobi integer variables.

Functions:
    search_materias_by_nombre(materias, nombre): Searches for subjects by name in the given list of subjects.

Workflow:
1. Populate `teo` and `vars1erHora` with IDs and variables for theoretical subjects.
2. Populate `prac` and `vars1erHora` with IDs and variables for practical subjects.
3. Create scaled variables `scaled_u_dict` for each subject and block combination.
4. Add constraints to the model to ensure that `vars1erHora` variables are the maximum of the scaled variables.
5. Add constraints to ensure that the first hour of theoretical subjects is greater than or equal to the first hour of practical subjects if the lengths of `teo` and `prac` are equal.


"""
prac = []    # array con IDS de materias de practico
vars1erHora = {}
scaled_u_dict = {}

for nombre in ["AM2", "ALN", "AM1", "AL", "Prob", "ProgAv", "ProgAv"]:
    # se carga ProgAv dos veces a proposito
    for m in search_materias_by_nombre(materias, nombre):
        teo.append(m.id)
        vars1erHora[m.id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"_1er_hora")

for nombre in ["AM2p", "ALNp", "AM1p", "ALp", "ProbP", "ProgAvP"]:
    for m in search_materias_by_nombre(materias, nombre):
        prac.append(m.id)
        vars1erHora[m.id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"_1er_hora")

L = len(bloques_horario_ids)

for m in teo + prac:
    for b in range(0, L):
        scaled_u_dict[m, bloques_horario_ids[b]] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"scaled_u")
        model.addConstr(scaled_u_dict[m, bloques_horario_ids[b]] == (L - b) * u_dict[m, bloques_horario_ids[b]].variable)

for m_id in vars1erHora:
    print(m_id, vars1erHora[m_id])
    model.addConstr(vars1erHora[m_id] ==
                    # gp.max_( (L - b) * u_dict[m_id, bloques_horario_ids[b]].variable
                    gp.max_( scaled_u_dict[m_id, bloques_horario_ids[b]]
                            for b in range(0, L)) )

if len(teo) == len(prac):
    model.addConstrs(vars1erHora[teo[m]] >= vars1erHora[prac[m]] for m in range(0, len(teo)))


48 <gurobi.Var *Awaiting Model Update*>
49 <gurobi.Var *Awaiting Model Update*>
52 <gurobi.Var *Awaiting Model Update*>
24 <gurobi.Var *Awaiting Model Update*>
25 <gurobi.Var *Awaiting Model Update*>
26 <gurobi.Var *Awaiting Model Update*>
27 <gurobi.Var *Awaiting Model Update*>
32 <gurobi.Var *Awaiting Model Update*>
33 <gurobi.Var *Awaiting Model Update*>
34 <gurobi.Var *Awaiting Model Update*>
35 <gurobi.Var *Awaiting Model Update*>
50 <gurobi.Var *Awaiting Model Update*>
51 <gurobi.Var *Awaiting Model Update*>
53 <gurobi.Var *Awaiting Model Update*>
28 <gurobi.Var *Awaiting Model Update*>
29 <gurobi.Var *Awaiting Model Update*>
30 <gurobi.Var *Awaiting Model Update*>
31 <gurobi.Var *Awaiting Model Update*>
36 <gurobi.Var *Awaiting Model Update*>
37 <gurobi.Var *Awaiting Model Update*>
38 <gurobi.Var *Awaiting Model Update*>
39 <gurobi.Var *Awaiting Model Update*>


### (3.4) Grupos

#### (3.4.1) definicion de variable "x"

$$ x_{gb} = \sum_{m \in M_g} {u_{mb}} $$



In [ ]:
for g in grupos:
    gr_mats_ids = materias_grupo_ids(g, materias)
    model.addConstrs(x_dict[g.id, b].variable == gp.quicksum(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)


#### (3.4.2) evitar horas puente por grupo

$$ \sum_h {x_{gdh}} - \sum_h {x_{gd(h)} · x_{gd(h+1)}} = 1 $$
    para todo g, d

In [ ]:
grupos_reales = [] # no considerar grupos ficticios ("dummy") o con carga horaria nula (a proposito)
for g in grupos:
    carga_horaria = 0
    for m in materias_grupo(g, materias):
        carga_horaria += m.carga_horaria
    if carga_horaria > 0:
        grupos_reales.append(g)

for g in grupos_reales:
    model.addConstrs(gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == 1 for d in dias_ids)

## (4) Funcion Objetivo

In [ ]:
# Set objective
OBJ = gp.QuadExpr()

### (4.1) Prioridad horaria de los docentes

$$ MIN: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [ ]:
OBJ1 = gp.QuadExpr()
"""
This script constructs a quadratic expression for an optimization problem using Gurobi's QuadExpr.
Variables:
    OBJ1 (gp.QuadExpr): A quadratic expression object to accumulate the terms.
    count (int): A counter to keep track of the number of terms added to the quadratic expression.
Loops:
    The script iterates over all combinations of 'materias' and 'profesores'. For each professor, it further iterates over their 'prioridades'.
Inner Loop:
    For each priority 'pr' of a professor:
        - 'b' is assigned the 'bloque_horario' of the priority.
        - 'A' is assigned the 'value' of the priority.
        - The quadratic expression 'OBJ1' is updated by adding the product of 'A', the variable corresponding to the combination of 'materia' and 'profesor' from 'w_dict', and the variable corresponding to the combination of 'materia' and 'bloque_horario' from 'u_dict'.
        - The counter 'count' is incremented.
Final Step:
    The constructed quadratic expression 'OBJ1' is added to the main objective 'OBJ'.
Output:
    The total count of terms added to the quadratic expression is printed.
"""

count = 0
for m in materias:
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable
            count += 1


OBJ += OBJ1

print(count)


9150


LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ MIN: \sum_{p,b} {A_{pb}·y_{pb}} $$

In [ ]:
# OBJ1 = gp.QuadExpr()

# count = 0
# for p in profesores:
#     for pr in p.prioridades:
#         b = pr.bloque_horario
#         A = pr.value
#         OBJ1 += A * y_dict[p.id, b.id()].variable
#         count += 1

# OBJ += OBJ1

# print(count)

### (4.2) Minimizar dias con clase por profesor
$$ MIN: \sum_{p,d} {OR_h{\sum_m{u_{mdh}·w_{mp}}}} $$
$$ MIN: \sum_{p,d} {z_{pd}} $$

In [ ]:
OBJ2 = gp.LinExpr()
DP = {}

for p in profesores:
    
    D_p = gp.quicksum(z_dict[p.id,d].variable for d in dias_ids)
    DP[p.id] = D_p

    # filtrar si el profesor prefiere minimizar o no
    if p.minimizar_dias:
        OBJ2 += D_p

P = 5

OBJ += P*OBJ2


### (4.3) Minimizar horas puente por grupo (alternativa a la restriccion, NO VA)

$$ MIN: -\sum_{g,d,h} {x_{gd(h)}·x_{gd(h+1)}} $$

In [ ]:
# OBJ3 = gp.QuadExpr()

# for g in grupos_reales:
#     for d in dias_ids:
#         OBJ3 += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
    
# # OBJ += OBJ3


## Resolver

In [ ]:
model.setObjective(OBJ, GRB.MINIMIZE)

In [ ]:
model.setParam("TimeLimit", 60*60*2)
model.optimize()


Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20736 rows, 29715 columns and 80917 nonzeros
Model fingerprint: 0x6ac1fcb1
Model has 12789 quadratic constraints
Model has 417 general constraints
Variable types: 0 continuous, 29715 integer (28043 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 1e+00]
Presolve removed 15733 rows and 23454 columns
Presolve time: 0.65s
Presolved: 20809 rows, 11088 columns, 68992 nonzeros
Variable types: 0 continuous, 11088 integer (10912 binary)

Root relaxation: objective 0.000000e+00, 19314 iterations, 3.11 second

In [ ]:

if model.Status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write('iismodel.ilp')

    # Print out the IIS constraints and variables
    print('\nThe following constraints and variables are in the IIS:')
    for c in model.getConstrs():
        if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

    for v in model.getVars():
        if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
        if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

### Imprimir horarios

In [ ]:
if not model.Status == GRB.INFEASIBLE:
    print('Obj: %g' % model.ObjVal)
    print('OBJ1: %g' % OBJ1.getValue())
    print('OBJ2: %g' % OBJ2.getValue())
    print('=============================================================')
    print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)

    copy_variables_excel(u_dict, w_dict)


Obj: 0
OBJ1: 5
OBJ2: 0

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		T_Ing		AL		---		Prog1		AM1
08:50-09:40		T_Ing		AL		AM1		Prog1		AM1
09:50-10:40		T_Ing		AL		AM1		T_Ing		AM1
10:40-11:30		Prog1		ALp		---		T_Ing		AL
11:40-12:30		Prog1		ALp		---		AM1p		AL
12:30-13:20		Prog1		---		---		AM1p		---

 Grupo:  12
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		---		---		AM1		ALp		AM1
08:50-09:40		---		AL		AM1		ALp		AM1
09:50-10:40		Prog1		AL		AM1p		Prog1		AM1
10:40-11:30		Prog1		T_Ing		AM1p		Prog1		AL
11:40-12:30		Prog1		T_Ing		---		T_Ing		AL
12:30-13:20		---		T_Ing		---		T_Ing		AL

 Grupo:  13
			lun		mar		mie		jue		vie
······················································

In [ ]:
# HORAS PUENTE

# HP = gp.QuadExpr()
# for g in grupos_reales:
#     HPg = gp.QuadExpr()
#     for d in dias_ids:
#         HPg += gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
#         HPg += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
#         HPg += - 1
#     # print("horas puente grupo", str(g), ": ", HPg.getValue())
#     HP += HPg

# print("Total de horas puente: ", HP.getValue())

In [ ]:
# DIAS CLASE POR PROFESOR

# print("DIAS CON CLASE")
# for p in profesores:
#     print(str(p), ": ", round(DP[p.id].getValue()))

In [ ]:
print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias)
# print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias, "ADA"))

# print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)


 Profesor:  aca
		lun	mar	mie	jue	vie
·····················································
08:00-08:50	---	MIE	---	---	---
08:50-09:40	---	MIE	---	---	---
09:50-10:40	---	MIE	---	---	---
10:40-11:30	---	---	---	---	---
11:40-12:30	---	---	---	---	---
12:30-13:20	---	---	---	---	---
14:10-15:00	---	---	---	---	---
15:10-16:00	---	---	---	---	---
16:10-17:00	---	---	---	---	---
17:00-17:50	---	---	---	---	---
18:00-18:50	---	---	---	---	---
18:50-19:40	---	---	---	---	---
19:50-20:40	---	---	---	---	---
20:40-21:30	---	---	---	---	---
21:40-22:30	---	---	---	---	---

 Profesor:  ad
		lun	mar	mie	jue	vie
·····················································
08:00-08:50	T_Ing	---	---	T_Ing	---
08:50-09:40	T_Ing	---	---	T_Ing	---
09:50-10:40	T_Ing	---	---	T_Ing	---
10:40-11:30	T_Ing	T_Ing	---	T_Ing	---
11:40-12:30	T_Ing	T_Ing	---	T_Ing	---
12:30-13:20	T_Ing	T_Ing	---	T_Ing	---
14:10-15:00	---	---	---	---	---
15:10-16:00	---	---	T_Ing	---	---
16:10-17:00	---	---	T_Ing	---	---
17:00-17:50	-

In [ ]:
# print_prof_timetable_excel(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "svv")], materias)
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "jk")])
# print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "jpa")], materias)
# print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "jb")], materias)

In [ ]:
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "gr")])
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "ig")])

In [ ]:
# imprimir calendario para profesores
# if not model.Status == GRB.INFEASIBLE:
    # print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "gc")], materias)
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias,"Prob"))
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias)

In [ ]:
# IMPRIMIR PARA COPIAR EN HOJA DE EXCEL
# print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, [anios[0]])

In [ ]:
# print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)

In [ ]:
# for m in materias:
#     print(str(m), ":", [str(g) for g in m.grupos])
    # print(m, [str(p) for p in m.profesores])

In [ ]:
# for p in profesores:
#     print(str(p), [str(m) for m in materias_profesor(p, materias)])

### Promedios de prioridad

In [ ]:
# CARGA_HORARIA_TOTAL = gp.quicksum(m.carga_horaria for m in materias)
CARGA_HORARIA_TOTAL = gp.quicksum(u_dict[m, b].variable * w_dict[m, p].variable for p in profesores_ids for m in materias_ids for b in bloques_horario_ids)

if not model.Status == GRB.INFEASIBLE:
    print(OBJ1.getValue())
    print("Promedio general de prioridad horaria: ", OBJ1.getValue()/CARGA_HORARIA_TOTAL.getValue())
          

# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


5.0
Promedio general de prioridad horaria:  0.01016260162601626


In [ ]:
# promedio por profesor:
"""
This script calculates and prints the average workload per professor based on their assigned priorities and time blocks.
Variables:
    profesores (list): List of professor objects, each containing their priorities.
    y_dict (dict): Dictionary mapping professor IDs and time block IDs to Gurobi variables.
    bloques_horario_ids (list): List of time block IDs.
For each professor in the list `profesores`, the script performs the following steps:
1. Initializes a quadratic expression `OBJ_p` to accumulate the weighted priorities.
2. Iterates over the professor's priorities to update `OBJ_p` based on the priority value and corresponding Gurobi variable.
3. Calculates the total workload `CARGA_HORARIA_p` for the professor by summing the relevant Gurobi variables.
4. If the total workload is non-zero, it prints the professor's name, total workload, and the rounded average priority value per workload unit.
5. If the total workload is zero, it prints the professor's name and indicates that they have no workload.

"""
for p in profesores:

    OBJ_p = gp.QuadExpr()
    # for m in materias:
    #     for pr in p.prioridades:
    #             b = pr.bloque_horario
    #             A = pr.value
    #             OBJ_p += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable

    for pr in p.prioridades:
        b = pr.bloque_horario
        A = pr.value
        OBJ_p += A * y_dict[p.id, b.id()].variable
    
    # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()
    CARGA_HORARIA_p = round(gp.quicksum(y_dict[p.id, b].variable for b in bloques_horario_ids).getValue())

    if CARGA_HORARIA_p != 0:
        print(str(p), "\t", CARGA_HORARIA_p, "\t", round(round(OBJ_p.getValue())/CARGA_HORARIA_p*1000))
    else:
        print(str(p), "sin carga horaria")

    


ac sin carga horaria
aca 	 3 	 0
ad 	 20 	 0
adr 	 4 	 0
am 	 12 	 0
ame 	 12 	 0
ap 	 4 	 0
apa 	 4 	 0
apr 	 3 	 0
as 	 20 	 0
cm 	 3 	 0
cp 	 10 	 0
cr 	 8 	 0
db 	 10 	 0
dc 	 9 	 0
dd 	 8 	 0
dj 	 4 	 0
dl 	 13 	 0
dm sin carga horaria
ds 	 13 	 0
dv sin carga horaria
ec 	 3 	 0
epe 	 6 	 0
es 	 3 	 0
fb 	 5 	 0
gaa 	 5 	 0
gb 	 11 	 0
gba 	 4 	 0
gc 	 12 	 0
gd 	 4 	 0
gq 	 5 	 0
gs sin carga horaria
guc 	 10 	 0
hf 	 12 	 0
hr 	 5 	 0
jam 	 8 	 0
jcc sin carga horaria
jd 	 18 	 0
jds 	 7 	 0
jf 	 8 	 0
jj 	 8 	 0
jk 	 14 	 0
jl 	 2 	 0
jpa 	 4 	 0
jpe 	 4 	 0
jpf 	 5 	 0
kb 	 5 	 0
lc 	 2 	 0
le sin carga horaria
lg 	 14 	 0
mb 	 5 	 0
mbo 	 4 	 0
mg 	 4 	 0
mjc 	 5 	 0
mp 	 5 	 0
mpi 	 5 	 0
mpr 	 4 	 0
mr 	 6 	 0
mro 	 5 	 0
mz 	 2 	 0
nd 	 6 	 0
nn 	 5 	 0
ns 	 8 	 0
pn 	 9 	 0
po 	 8 	 0
ps 	 8 	 0
pu 	 4 	 0
rc 	 3 	 0
rl 	 5 	 0
rr 	 3 	 0
rs 	 6 	 0
sb 	 5 	 0
sgp 	 5 	 0
sp 	 5 	 0
st sin carga horaria
svi 	 3 	 0
svv 	 10 	 0
th 	 5 	 0
vt 	 5 	 1000


In [ ]:
# distribucion de prioridades
"""
This script calculates and prints the distribution of priorities and their averages for a given set of professors and their time block priorities.
Variables:
    promedios_prioridad (dict): A dictionary to store the average priority values for each priority level.
    valores_prioridad (dict): A dictionary to store the count of each priority level.
    profesores (list): A list of professor objects, each containing their priority information.
    y_dict (dict): A dictionary containing the decision variables for the optimization model.
Process:
1. Initialize dictionaries to store priority averages and counts.
2. Iterate over each professor and their priorities:
    - Skip if the priority value is 0.
    - Increment the count for the given priority value.
    - Update the average priority value based on the decision variable.
3. Print the count and percentage distribution of each priority level.
4. Print the average priority values and their percentage distribution.

"""

promedios_prioridad = {}
valores_prioridad = {}
for i in range(1,4):
    promedios_prioridad[i] = 0
    valores_prioridad[i] = 0

for p in profesores:
    for pr in p.prioridades:
        b = pr.bloque_horario
        A = pr.value
        if A == 0:
            continue
        valores_prioridad[A] += 1
        # for m in materias:
        #     promedios_prioridad[A] += round(w_dict[m.id, p.id].variable.X * u_dict[m.id, b.id()].variable.X)
        promedios_prioridad[A] += round(y_dict[p.id, b.id()].variable.X)

print(valores_prioridad)
total = sum([valores_prioridad[i] for i in valores_prioridad])
for i in valores_prioridad:
    print(i, ": ", str(round(valores_prioridad[i]/total*100))+"%")

print(promedios_prioridad)
total = sum([promedios_prioridad[i] for i in promedios_prioridad])
for i in promedios_prioridad:
    if total != 0:
        print(i, ": ", str(round(promedios_prioridad[i]/total*100))+"%")
    
    # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()

    # if CARGA_HORARIA_p != 0:
    #     print(str(p), ": ", OBJ_p.getValue()/CARGA_HORARIA_p)
    # else:
    #     print(str(p), "sin carga horaria")

{1: 75, 2: 0, 3: 0}
1 :  100%
2 :  0%
3 :  0%
{1: 5, 2: 0, 3: 0}
1 :  100%
2 :  0%
3 :  0%


### Ocupacion de salones

In [ ]:
# nivel de ocupacion
niveles = {}

for b_id in bloques_horario:
    salones_ocupados = int(gp.quicksum(u_dict[m, b_id].variable for m in materias_ids).getValue())
    nivel = int(salones_ocupados/K*100)
    # print(str(bloques_horario[b_id]), ": ", salones_ocupados,",", str(nivel) + "%")
    niveles[b_id] = nivel

print_timetable_salones(dias, horarios, niveles)

		lun	mar	mie	jue	vie
··········································································································
08:00-08:50	53%	46%	53%	69%	53%
08:50-09:40	53%	69%	76%	76%	69%
09:50-10:40	69%	76%	76%	76%	76%
10:40-11:30	76%	84%	76%	84%	76%
11:40-12:30	92%	84%	61%	84%	76%
12:30-13:20	84%	61%	61%	61%	53%
14:10-15:00	7%	0%	0%	7%	15%
15:10-16:00	15%	30%	23%	23%	23%
16:10-17:00	15%	30%	23%	38%	30%
17:00-17:50	15%	15%	30%	38%	46%
18:00-18:50	53%	46%	61%	61%	69%
18:50-19:40	76%	61%	61%	61%	76%
19:50-20:40	69%	38%	38%	46%	61%
20:40-21:30	46%	30%	38%	30%	53%
21:40-22:30	15%	7%	30%	15%	30%


In [ ]:
# for u_i in u_dict:
#     u = u_dict[u_i]
#     print(str(u) + " = ", u.variable.X)

# for v_i in v_dict:
#     v = v_dict[v_i]
#     print(str(v) + " = ", v.variable.X)

# for w_i in w_dict:
#     w = w_dict[w_i]
#     print(str(w) + " = ", w.variable.X)


In [ ]:
model.write('timetable_opt.lp')
model.write('timetable_opt.mps')

# model.display()